## Building Docker Artifacts

In [25]:
cd pipeline_components

/home/melnamoury/Desktop/kubeflow-sdk-demo/pipeline_components


In [27]:
!docker login -u username -p password

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/melnamoury/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [28]:
%%writefile config.py
Bucket_uri="Bucket URI"
version=1
store_artifacts=Bucket_uri + "/" + str(version)
data_path=Bucket_uri + "/" + "data/train.csv"
processed_data=Bucket_uri + "/" + "processed/data_processed.csv"

Writing config.py


In [46]:
%%writefile Dockerfile
FROM gcr.io/deeplearning-platform-release/tf-cpu
ARG DEBIAN_FRONTEND=noninteractive
# Install apt dependencies
WORKDIR /pipeline
COPY ./ ./
RUN pip install -r requirements.txt
RUN pip install "dask[dataframe]" --upgrade
ENV TF_CPP_MIN_LOG_LEVEL 3

Overwriting Dockerfile


## Build Docker Image

In [47]:
!docker build -t melnamoury/kubeflow-sdk-farmer .

Sending build context to Docker daemon  4.657MB
Step 1/7 : FROM gcr.io/deeplearning-platform-release/tf-cpu
 ---> 20d85d5ff60a
Step 2/7 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> a8595af4abb3
Step 3/7 : WORKDIR /pipeline
 ---> Running in 811678c3b1a2
Removing intermediate container 811678c3b1a2
 ---> 89afbec689bd
Step 4/7 : COPY ./ ./
 ---> 496e65e8bfd8
Step 5/7 : RUN pip install -r requirements.txt
 ---> Running in 5edfa81b3820
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.8 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=f6fdddb1b3245d1c4f6e6ebf681c13b34e87248866c9afa647115ae9e72d387c
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3

## Push Docker Image to Docker hub

In [48]:
!docker push melnamoury/kubeflow-sdk-farmer

The push refers to repository [docker.io/melnamoury/kubeflow-sdk-farmer]

a508d677: Preparing 
eee8f0b9: Preparing 
9fde6097: Preparing 
81ed4975: Preparing 
daf002b4: Preparing 
99a0501a: Preparing 
d66050d7: Preparing 
bcab0add: Preparing 
a316c5c6: Preparing 
198535a8: Preparing 
41727f83: Preparing 
5878c855: Preparing 
78b45d85: Preparing 
198535a8: Waiting g 
59494948: Preparing 
78b45d85: Waiting g 
5878c855: Waiting g 
52904e20: Preparing 
5965a208: Waiting g 
8c5c8a78: Preparing 
59494948: Waiting g 
5afcf21f: Preparing 
66050d7: Pushing  78.97MB/3.222GB^C


## Install Kubeflow SDk

In [36]:
!pip install kfp

# Building Kubeflow Pipeline

In [37]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

## Define kubeflow pipeline Component

In [38]:
@kfp.dsl.component
def get_data():
    # Defining component configuration
    getdata_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='docker.io/ankitaj86/kubeflow-sdk-farmer',
        command=['python', 'get_data.py'],
        )
    return getdata_component

In [39]:
@kfp.dsl.component
def data_preprocessing():    
    # Defining component configuration
    data_preprocessing = kfp.dsl.ContainerOp(
        name='data-preprocessing',
        image='docker.io/ankitaj86/kubeflow-sdk-farmer',
        command=['python', 'process_data.py'],
        )
    return data_preprocessing

In [40]:
@kfp.dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='docker.io/ankitaj86/kubeflow-sdk-farmer',
        command=['python', 'train.py'],
        file_outputs={'mlpipeline-ui-metadata':'/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    
    return training_component

In [41]:
# Let see output of component configuration
debug = True
if debug :
    training_component_vis = training()
    print(training_component_vis)

{'ContainerOp': {'is_exit_handler': False, 'human_name': 'training', 'display_name': None, 'name': 'training 800007f7074a6f5b', 'node_selector': {}, 'volumes': [], 'tolerations': [], 'affinity': {}, 'pod_annotations': {}, 'pod_labels': {}, 'num_retries': 0, 'retry_policy': None, 'backoff_factor': None, 'backoff_duration': None, 'backoff_max_duration': None, 'timeout': 0, 'init_containers': [], 'sidecars': [], 'loop_args': None, '_component_spec_inputs_with_pipeline_params': [], '_inputs': [], 'dependent_names': [], 'enable_caching': True, 'attrs_with_pipelineparams': ['node_selector', 'volumes', 'pod_annotations', 'pod_labels', 'num_retries', 'init_containers', 'sidecars', 'tolerations', '_container', 'artifact_arguments', '_parameter_arguments'], '_is_v2': False, '_container': {'args': None,
 'command': ['python', 'train.py'],
 'env': None,
 'env_from': None,
 'image': 'docker.io/ankitaj86/kubeflow-sdk-farmer',
 'image_pull_policy': None,
 'lifecycle': None,
 'liveness_probe': None,
 

/home/melnamoury/anaconda3/envs/hug_face/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1257: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(


## Final Kubeflow pipeline Definition

In [43]:
@kfp.dsl.pipeline(
  name="Modeling simple sentiment Analysis pipeline",
  description="First intilaized pipeline to run over Kubeflow"
)
def kfp_pipline():
    download_data = get_data()
    download_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    data_processing = data_preprocessing().after(download_data)
    data_processing.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train = training().after(data_processing)
    train.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [ ]:
# Let see output of pipeline configuration
debug = True
if debug :
    training_pipeline_output = kfp_pipline()
    print(training_pipeline_output)

## Compile Kubeflow Pipeline 
* It will Generate .zip file inside this contain YAMl file which contain the configuration of kubeflow pipeline

In [ ]:
kfp.compiler.Compiler().compile(kfp_pipline, 'farmer-kubeflow-sdk-demo.zip')

## Connect to deployed kubeflow pipeline Endpoint (GCP)

In [ ]:
# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
client = kfp.Client("57f227181c3ac171-dot-us-central1.pipelines.googleusercontent.com")

### Create Experiment 

In [ ]:
EXPERIMENT_NAME = 'First-Pipeline-1'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

### Deploy pipeline to kubeflow pipeline Endpoint

In [ ]:
run = client.run_pipeline(experiment.id, 'First-run-3', 'Sentiment-kubeflow-sdk-demo.zip')